In [ ]:
import socket
import pickle
import sklearn
import time
import numpy as np
import struct

In [ ]:
dtree = '/home/admin2/ML_NIC/dtree/cicids/model.pkl'

In [ ]:
# Load up model
with open(dtree, 'rb') as f:
    clf = pickle.load(f)

In [ ]:
sock = socket.socket(socket.AF_PACKET, socket.SOCK_RAW, socket.htons(0x4d49))
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('enp2s0np1', 0))

In [ ]:
times_1 = []
times_2 = []

In [ ]:

# Run a dummy to take out the total load and dump time
while (True):
    data, addr = sock.recvfrom(2048)
    
#     start_1 = time.time_ns()
    ethHdr = data[0:14]
    ethH = struct.unpack('!6s6s2s', ethHdr)
    
    ipHdr = data[14:34]
    ipH = struct.unpack('!12s4s4s', ipHdr)
    
    ipProto = ipHdr[9]
    ipFlags = ipHdr[6] >> 5
    tcpDst = 0
    udpDst = 0
    pktSize = int.from_bytes(ipHdr[2:4])
    tcpFlags = 0
    
    l4Hdr = None
    if (ipProto == 6):
        l4Hdr = data[34:54]
        tcpDst = int.from_bytes(l4Hdr[2:4])
        tcpFlags = l4Hdr[13] & 0x3f
        
    elif (ipProto == 17):
        l4Hdr = data[34:42]
        udpDst = int.from_bytes(l4Hdr[2:4])
        
    elif (ipProto == 1 or ipProto == 2):
        l4Hdr = data[34:38]
        
    
    # feature_names = ['etherType', 'ipProto', 'ipFlags', 'tcpDst', 'udpDst', 'pktSize', 'tcpFlags']
    dataIn = np.array([[2048, ipProto, ipFlags, tcpDst, udpDst, pktSize, tcpFlags]])
#     end_1 = time.time_ns()
    
    pred = clf.predict(dataIn)
        
#     start_2 = time.time_ns()
    pred = int(pred[0]).to_bytes(1)
    out_pkt = ethH[1] + ethH[0] + b'\x4d\x4f' + ipH[0] + ipH[2] + ipH[1] + l4Hdr + pred
#     end_2 = time.time_ns()
    
    sock.send(out_pkt)
    
#     times_1.append(end_1 - start_1)
#     times_2.append(end_2 - start_2)

In [ ]:
sock.close()

In [ ]:
print(np.average(times_1))
print(np.average(times_2))